### SFT & GRPO on qwen3 model with unsloth 

- Dataset: OpenMathReasoning-mini 
- infer FM: unsloth/Qwen3-4B-Base 
- SFT with LoRA 
- infer SFT-model
- GRPO with LoRA
- infer GRPO model 

In [1]:
from unsloth import FastLanguageModel
import torch

from datasets import load_dataset
from datasets import Dataset

import pandas as pd
import numpy as np

from trl import SFTTrainer, SFTConfig
from trl import GRPOConfig, GRPOTrainer

from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-25 21:56:18.180011: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-25 21:56:18.302813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-25 21:56:18.360393: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-25 21:56:18.374977: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-25 21:56:18.441269: I tensorflow/core/platform/cpu_feature_guar

🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-25 21:56:26 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-25 21:56:26 [__init__.py:239] Automatically detected platform cuda.


In [2]:
!pip list | grep accelerate

accelerate                               1.1.0


In [3]:
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

In [4]:
!nvidia-smi

Sun May 25 21:56:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   40C    P0             28W /   70W |     103MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Load model

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.33%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.15 GB. Also swap space = 0 GB.
WARNING 05-25 21:56:35 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-25 21:56:47 [config.py:717] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. De

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 05-25 21:56:58 [loader.py:458] Loading weights took 8.35 seconds
INFO 05-25 21:56:58 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-25 21:56:58 [model_runner.py:1140] Model loading took 7.6334 GiB and 9.024566 seconds
INFO 05-25 21:57:02 [worker.py:287] Memory profiling takes 3.64 seconds
INFO 05-25 21:57:02 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.57GiB) x gpu_memory_utilization (0.69) = 10.10GiB
INFO 05-25 21:57:02 [worker.py:287] model weights take 7.63GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.88GiB; the rest of the memory reserved for KV Cache is 1.56GiB.
INFO 05-25 21:57:03 [executor_base.py:112] # cuda blocks: 709, # CPU blocks: 0
INFO 05-25 21:57:03 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 5.54x
INFO 05-25 21:57:03 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, 

Capturing CUDA graph shapes:   0%|          | 0/23 [00:00<?, ?it/s]

INFO 05-25 21:57:30 [model_runner.py:1592] Graph capturing finished in 28 secs, took 0.34 GiB
INFO 05-25 21:57:30 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 32.23 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth 2025.5.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [7]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

In [8]:
system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
print(system_prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>


In [9]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

In [10]:
# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [11]:
test_prompt = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

print(test_prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>


### Load data

In [12]:
dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

In [13]:
dataset

,expected_answer,problem,generated_solution
0,14,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ..."
1,convergent,Let \( \sum a_n \) be a convergent series with...,"<think>\nOkay, so I have this problem here whe..."
2,\(\frac{\pi \ln a}{2a}\),"For \( a > 0 \), find the value of \( \int_0^{...","<think>\nOkay, so I need to solve the integral..."
3,\(+\infty\),Calculate $\lim_{n\to\infty}\sqrt[n]{n!}$.,"<think>\nOkay, so I need to find the limit as ..."
4,\( n = 1 \),Find all positive integers \( n \) such that \...,"<think>\nOkay, so I need to find all positive ..."
...,...,...,...
19247,4,A bus left point X for point Y. Two hours late...,"<think>\nOkay, let's tackle this problem step ..."
19248,18,Each interior angle of a regular n-gon measure...,"<think>\nOkay, let's see. I need to find the n..."
19249,\( e^3 - 1 \),Evaluate the series \( \sum_{k=1}^{\infty}\fra...,"<think>\nOkay, so I need to evaluate the serie..."
19250,0.8960,Find the probability that the second blue resu...,"<think>\nOkay, so I need to find the probabili..."


In [14]:
# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]
dataset

,expected_answer,problem,generated_solution
0,14,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ..."
6,-2,Find the value of the parameter $a$ for which ...,"<think>\nOkay, so I need to find the value of ..."
9,18,What is the sum of all real numbers $x$ for wh...,"<think>\nOkay, so I need to solve the equation..."
13,2,Evaluate the sum \(\sum_{n=1}^\infty \frac{\ph...,"<think>\nOkay, so I need to evaluate the infin..."
17,30,What is the largest positive integer that divi...,"<think>\nAlright, so I need to find the larges..."
...,...,...,...
19243,244,"Let \( p \), \( q \), and \( r \) be the disti...","<think>\nOkay, so I need to find the value of ..."
19245,1,A bug is on the $0$ of a number line. At any p...,"<think>\nOkay, so I have this problem where a ..."
19247,4,A bus left point X for point Y. Two hours late...,"<think>\nOkay, let's tackle this problem step ..."
19248,18,Each interior angle of a regular n-gon measure...,"<think>\nOkay, let's see. I need to find the n..."


Split train_set and test_set for SFT (Instruction-Tuning)

In [15]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset,random_state=42,test_size=0.2)
print(train_set.shape, test_set.shape)

(6005, 3) (1502, 3)


Data formatting and filtering

In [17]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    # Remove generated <think> and </think>
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

In [18]:
train_set["Messages"] = train_set.apply(format_dataset, axis = 1)
train_set = train_set.reset_index(drop=True)
train_set["Messages"]

0       [{'role': 'system', 'content': 'You are given ...
1       [{'role': 'system', 'content': 'You are given ...
2       [{'role': 'system', 'content': 'You are given ...
3       [{'role': 'system', 'content': 'You are given ...
4       [{'role': 'system', 'content': 'You are given ...
                              ...                        
6000    [{'role': 'system', 'content': 'You are given ...
6001    [{'role': 'system', 'content': 'You are given ...
6002    [{'role': 'system', 'content': 'You are given ...
6003    [{'role': 'system', 'content': 'You are given ...
6004    [{'role': 'system', 'content': 'You are given ...
Name: Messages, Length: 6005, dtype: object

In [19]:
test_set["Messages"] = test_set.apply(format_dataset, axis = 1)
test_set = test_set.reset_index(drop=True)
test_set["Messages"]

0       [{'role': 'system', 'content': 'You are given ...
1       [{'role': 'system', 'content': 'You are given ...
2       [{'role': 'system', 'content': 'You are given ...
3       [{'role': 'system', 'content': 'You are given ...
4       [{'role': 'system', 'content': 'You are given ...
                              ...                        
1497    [{'role': 'system', 'content': 'You are given ...
1498    [{'role': 'system', 'content': 'You are given ...
1499    [{'role': 'system', 'content': 'You are given ...
1500    [{'role': 'system', 'content': 'You are given ...
1501    [{'role': 'system', 'content': 'You are given ...
Name: Messages, Length: 1502, dtype: object

In [20]:
train_set["Messages"][0][:2]

[{'role': 'system',
  'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'},
 {'role': 'user',
  'content': 'Consider the sequence of numbers: $4, 7, 1, 8, 9, 7, 6, \\ldots$. For $n > 2$, the $n$th term of the sequence is the units digit of the sum of the two previous terms. Let $S_n$ denote the sum of the first $n$ terms of this sequence. What is the smallest value of $n$ for which $S_n > 10,000$?'}]

In [21]:
test_text = tokenizer.apply_chat_template(train_set["Messages"][0], tokenize = False)
print(test_text)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Consider the sequence of numbers: $4, 7, 1, 8, 9, 7, 6, \ldots$. For $n > 2$, the $n$th term of the sequence is the units digit of the sum of the two previous terms. Let $S_n$ denote the sum of the first $n$ terms of this sequence. What is the smallest value of $n$ for which $S_n > 10,000$?<start_working_out>Okay, let me try to figure out this problem. So, we have this sequence that starts with 4, 7, and then each next term is the units digit of the sum of the two previous terms. We need to find the smallest n where the sum of the first n terms exceeds 10,000. Hmm, let's break this down.

First, let me write out the sequence as given and continue it for a few more terms to see if there's a pattern. The sequence starts with 4, 7. The third term is the units digit of 4 + 7, which is

In [26]:
# select dataset shorter than max_seq_length/2
train_set["N"] = train_set["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))
train_set = train_set.loc[train_set["N"] <= max_seq_length/2].copy()
train_set.shape

(47, 5)

In [27]:
# convert to dataset format 
train_set["text"] = tokenizer.apply_chat_template(train_set["Messages"].values.tolist(), tokenize = False)
train_set = Dataset.from_pandas(train_set)
train_set.shape

(47, 7)

In [24]:
# select dataset shorter than max_seq_length/2
test_set["N"] = test_set["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))
test_set = test_set.loc[test_set["N"] <= max_seq_length/2].copy()
test_set.shape

(12, 5)

In [25]:
# convert to dataset format 
test_set["text"] = tokenizer.apply_chat_template(test_set["Messages"].values.tolist(), tokenize = False)
test_set = Dataset.from_pandas(test_set)
test_set.shape

(12, 7)

### Inference on Foundation Model

In [28]:
IDX = 2

In [29]:
test_set[IDX]['problem'], test_set[IDX]['expected_answer']

('In how many different ways can 14 identical cans of tennis balls be sold over three days (Friday, Saturday, and Sunday)?',
 '120')

In [31]:
prompt = tokenizer.apply_chat_template(
    test_set[IDX]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

print(prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>In how many different ways can 14 identical cans of tennis balls be sold over three days (Friday, Saturday, and Sunday)?<start_working_out>


In [33]:
# Method 1: fast_generate()
response = model.fast_generate(
    prompt,
    # Add other generation parameters if required
)[0]

generated_text = response.outputs[0].text

print(generated_text)

WARNING 05-25 22:03:45 [config.py:1239] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 
To solve this problem, we can use the method of stars and bars, which is a combinatorial technique used to determine the number of ways to distribute indistinguishable objects into distinguishable bins. 

In this case, we have 14 identical cans of tennis balls to be sold over three days (Friday, Saturday, and Sunday). We can represent this problem as finding the number of non-negative integer solutions to the equation: x + y + z = 14, where x represents the number of cans of tennis balls sold on Friday, y represents the number sold on Saturday, and z represents the number sold on Sunday.

To apply the stars and bars method, we need to find the number of ways to arrange 14 stars (representing the 14 cans of tennis balls) and 2 bars (which separate the three days).

There are a total of 16 positions (14 stars + 2 bars), and we need to select 2 positions for the bars out of the 16 positions available. This can be calculated using the combination formula:

C(n, k) = n! / (k!(n-k)!)

wher

In [32]:
# Method 2: generate()
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU if available

outputs = model.generate(
    **inputs,
    temperature = 0,
    max_new_tokens=2048,  # Adjust as needed
    # Add other generation parameters if required
)[0]

generated_text = tokenizer.decode(outputs, skip_special_tokens=True)
print(generated_text)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>In how many different ways can 14 identical cans of tennis balls be sold over three days (Friday, Saturday, and Sunday)?<start_working_out> 
To solve this problem, we can use the stars and bars method. We have 14 identical cans of tennis balls to distribute over three days (Friday, Saturday, and Sunday). We can represent the distribution of cans as a sequence of 14 stars (representing the cans) and 2 bars (representing the divisions between the days). For example, the sequence "****|****|****" represents 4 cans on Friday, 4 cans on Saturday, and 6 cans on Sunday.

The number of ways to arrange the stars and bars is given by the binomial coefficient:

C(n + k - 1, k - 1)

where n is the number of identical items (cans) and k is the number of categories (days). In this case, n = 14 and k = 3.

C

In [35]:
# Method 3: text_streamer
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU if available

_ = model.generate(
    **inputs,
    temperature = 0,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Sunday)?<start_working_out> between <SOLUTION></SOLUTION><|endoftext|>In how many different ways can 14 identical cans of tennis balls be sold over three days (Friday, Saturday, and 
To solve this problem, we can use the stars and bars method. We have 14 identical cans of tennis balls to distribute over three days (Friday, Saturday, and Sunday). We can represent the distribution of cans as a sequence of 14 stars (representing the cans) and 2 bars (representing the divisions between the days). For example, the sequence "****|****|****" represents 4 cans on Friday, 4 cans on Saturday, and 6 cans on Sunday.

The number of ways to arrange the stars and bars is given by the binomial coefficient:

C(n + k - 1, k - 1)

where n is the number of identical items (cans) and k is the number of categories (days). In this case, n = 14 and k = 3.

C(14 + 3 - 1, 3 

### SFT

In [36]:
OUTPUT_DIR = "openmathreasoning_qwen3_sft"

In [37]:
sft_trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 20,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        output_dir = OUTPUT_DIR,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/47 [00:00<?, ? examples/s]

In [38]:
train_result = sft_trainer.train()

metrics = train_result.metrics
sft_trainer.log_metrics("train", metrics)
sft_trainer.save_metrics("train", metrics)
sft_trainer.save_state()
print(metrics)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47 | Num Epochs = 10 | Total steps = 470
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


Step,Training Loss
20,0.539700
40,0.418700
60,0.302800
80,0.221800
100,0.211800
120,0.123500
140,0.122700
160,0.076600
180,0.067700
200,0.061600


Unsloth: Will smartly offload gradients to save VRAM!
***** train metrics *****
  total_flos               =  8786426GF
  train_loss               =     0.1013
  train_runtime            = 0:10:08.49
  train_samples_per_second =      0.772
  train_steps_per_second   =      0.772
{'train_runtime': 608.4949, 'train_samples_per_second': 0.772, 'train_steps_per_second': 0.772, 'total_flos': 9434353119866880.0, 'train_loss': 0.10130086479351876}


### Inference after SFT

In [39]:
IDX = 10

In [40]:
test_set[IDX]['problem'], test_set[IDX]['expected_answer']

('Find the probability that the game between Nicole and Maria will last only two sets, given that the first player to win 2 sets wins the match and the probability that Nicole wins any set is 70%.',
 '0.58')

In [41]:
prompt = tokenizer.apply_chat_template(
    test_set[IDX]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

print(prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Find the probability that the game between Nicole and Maria will last only two sets, given that the first player to win 2 sets wins the match and the probability that Nicole wins any set is 70%.<start_working_out>


In [43]:
# Method 1: fast_generate()
response = model.fast_generate(
    prompt,
    # Add other generation parameters if required
)[0]

generated_text = response.outputs[0].text

print(generated_text)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 
To find the probability that the game between Nicole and Maria will last only two sets, we need to consider the scenario where the first set is won by Nicole, and the second set is then won by Maria. We can also consider the scenario where the first set is won by Maria, and the second set is then won by Nicole.

Given that the probability Nicole wins any set is 0.7, the probability that Nicole wins the first set is 0.7, and since there are only two possible outcomes for each set (either Nicole wins or Maria wins), the probability that Maria wins the second set in this scenario is 0.3 (1 - 0.7).

Similarly, the probability that Maria wins the first set is 1 - 0.7 = 0.3, and if Maria wins the first set, the probability that Nicole wins the second set is still 0.7.

Now we can calculate the probability of the game lasting only two sets by adding the probabilities of these two scenarios:

1. Nicole wins the first set, and Maria wins the second set: \(0.7 * 0.3 = 0.21\)
2. Maria wins the 

In [44]:
# Method 2: generate()
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU if available

outputs = model.generate(
    **inputs,
    temperature = 0,
    max_new_tokens=2048,  # Adjust as needed
    # Add other generation parameters if required
)[0]

generated_text = tokenizer.decode(outputs, skip_special_tokens=True)
print(generated_text)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>Find the probability that the game between Nicole and Maria will last only two sets, given that the first player to win 2 sets wins the match and the probability that Nicole wins any set is 70%.<start_working_out>Okay, so I need to find the probability that the game between Nicole and Maria lasts only two sets. The condition is that the first player to win two sets wins the match. Nicole has a 70% chance of winning any set, and Maria would then have a 30% chance.

Let me think. If the game lasts only two sets, that means one of the players must have won both the first and the second set. Because if each had won one set, they'd have to play a third one to determine the winner. So the match ending in two sets would happen if either Nicole wins both sets or Maria wins both sets.

So the probabili

In [45]:
# Method 3: text_streamer
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU if available

_ = model.generate(
    **inputs,
    temperature = 0,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
70%.<start_working_out>Okay, so I need to find the probability that the game between Nicole and Maria lasts only two sets. The condition is that the first player to win two sets wins the match. Nicole has a 70% chance of winning any set, and Maria would then have a 30% chance.

Let me think. If the game lasts only two sets, that means one of the players must have won both the first and the second set. Because if each had won one set, they'd have to play a third one to determine the winner. So the match ending in two sets would happen if either Nicole wins both sets or Maria wins both sets.

So the probability we're looking for is the sum of the probability that Nicole wins both sets and the probability that Maria wins both sets.

Given that each set is independent, the probability of Nicole winning two in a row would be 0.7 * 0.7, right? And Maria's

### GRPO

In [46]:
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text

In [48]:
extract_hash_answer(test_set[10]["expected_answer"])

'0.58'

In [49]:
print(system_prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>


Prepare GRPO dataset

In [65]:
grpo_dataset = train_set.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["problem"]},
    ],
    "answer": extract_hash_answer(x["expected_answer"]),
})
grpo_dataset[0]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

{'expected_answer': '48',
 'problem': 'How many subsets of the set {A, B, C, D, E, F} contain at least one of the letters E or F?',
 'generated_solution': '<think>\nOkay, so I need to find how many subsets of the set {A, B, C, D, E, F} contain at least one of the letters E or F. Hmm, let me think. \n\nFirst, I remember that the total number of subsets of a set with n elements is 2^n. Since there are 6 elements here, the total subsets would be 2^6. Let me calculate that. 2^6 is 64. So there are 64 subsets in total.\n\nBut the problem asks for subsets that contain at least one of E or F. That means I need to exclude the subsets that have neither E nor F. Wait, maybe it\'s easier to calculate the total subsets and then subtract the subsets that don\'t have E or F. Yeah, that makes sense. Because "at least one" often is calculated using the complement.\n\nSo, how many subsets don\'t contain E or F? Well, if we remove E and F from the original set, we\'re left with {A, B, C, D}. The number 

Define GRPO reward functions

In [66]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|endoftext\|>)?[\s]{0,}$',
           re.MULTILINE|re.DOTALL|re.UNICODE)

In [67]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [68]:
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION>  2  </SOLUTION>\n\n",
)

['  2  ']

In [69]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [70]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

In [71]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # Correct answer gets 5 points!
        if guess == true_answer:
            score += 5.0
        # Match if spaces are seen, but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # Penalize wrong answers
            except:
                score -= 4.5 # Penalize
        scores.append(score)
    return scores

In [72]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

['0.34']
['123,456']
['-0.234']
['17']


In [73]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 1

In [74]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

In [75]:
tokenized = grpo_dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>How many subsets of the set {A, B, C, D, E, F} contain at least one of the letters E or F?<start_working_out>


Map:   0%|          | 0/47 [00:00<?, ? examples/s]

In [76]:
import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

Max Length =  104


Final dataset for GRPO training - must contain 'prompt' feature

In [77]:
# Filter only samples smaller than 90% max length
grpo_dataset = grpo_dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
grpo_dataset
#del tokenized

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'N', 'text', '__index_level_0__', 'prompt', 'answer'],
    num_rows: 42
})

GRPO training

In [79]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

In [80]:
training_args = GRPOConfig(
    #vllm_sampling_params = vllm_sampling_params,
    temperature = 0.01,   #1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 50,   #100,
    save_steps = 50,  #100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs_grpo_qwen",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 2


In [82]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = grpo_dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)

In [83]:
train_result = trainer.train()

metrics = train_result.metrics
sft_trainer.log_metrics("train", metrics)
sft_trainer.save_metrics("train", metrics)
sft_trainer.save_state()
print(metrics)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 42 | Num Epochs = 2 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 66,060,288/4,088,528,384 (1.62% trained)


********************Question:
In how many different ways can 14 identical cans of tennis balls be sold over three days (Friday, Saturday, and Sunday)? 
Answer:
120 
Response:
Okay, so I need to figure out how many different ways 14 identical cans of tennis balls can be sold over three days: Friday, Saturday, and Sunday. Hmm, let me think. Since the cans are identical, the order of the cans sold each day doesn't matter, right? It's more about distributing the identical items into different bins (days). So this seems like a combinatorics problem, specifically a "stars and bars" scenario.

The stars and bars theorem is used to find the number of ways to distribute identical objects into distinct groups. The formula for distributing n identical items into k distinct groups is (n + k - 1) choose (k - 1). Let me confirm that. Yeah, the formula is C(n + k -1, k -1) where C is the combination function. 

In this case, n is 14 cans, and k is 3 days. So plugging into the formula, that would be C

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,0.014200,13.000000,0.000000,915.000000,0.355486,3.000000,1.500000,5.000000,3.500000
2,0.016500,13.000000,0.000000,936.000000,0.412660,3.000000,1.500000,5.000000,3.500000
3,0.012200,13.000000,0.000000,638.000000,0.305328,3.000000,1.500000,5.000000,3.500000
4,0.012200,13.000000,0.000000,916.000000,0.305548,3.000000,1.500000,5.000000,3.500000
5,0.015500,13.000000,0.000000,744.000000,0.386921,3.000000,1.500000,5.000000,3.500000
6,0.009500,13.000000,0.000000,619.000000,0.238283,3.000000,1.500000,5.000000,3.500000
7,0.012400,13.000000,0.000000,883.000000,0.309099,3.000000,1.500000,5.000000,3.500000
8,0.010900,13.000000,0.000000,915.000000,0.271775,3.000000,1.500000,5.000000,3.500000
9,0.010700,13.000000,0.000000,825.000000,0.267549,3.000000,1.500000,5.000000,3.500000
10,0.011800,13.000000,0.000000,841.000000,0.294758,3.000000,1.500000,5.000000,3.500000


********************Question:
In how many ways can 5 persons be seated in a circle with only 3 chairs available? 
Answer:
20 
Response:
Okay, so I need to figure out how many ways 5 people can be seated in a circle with only 3 chairs available. Hmm, let's see. This is a permutation problem, but since it's in a circle, the arrangement is circular. Wait, but how does that work with only some chairs? Maybe I should start by recalling the formula for circular permutations.

For circular permutations of n people, the formula is (n-1)! because rotating the circle doesn't create a new arrangement. But here, we have more people than chairs. So it's like choosing 3 people out of 5 and then arranging them in a circle.

So maybe first, I need to compute the number of ways to choose 3 people from 5. That would be the combination part: C(5,3). Then, multiply by the number of ways to arrange those 3 people in a circle, which is (3-1)! = 2!.

Let me check that. If we had 3 chairs and 3 people, the nu

In [85]:
!nvidia-smi

Sun May 25 23:12:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   41C    P0             28W /   70W |   14519MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Inference after GRPO

In [86]:
IDX = 9

In [87]:
test_set[IDX]['problem'], test_set[IDX]['expected_answer']

('Evaluate \\(1 + 2 + 3 + 4 + 5 + 7 + 8 + 9 + 10 \\times 0\\).', '39')

In [88]:
prompt = tokenizer.apply_chat_template(
    test_set[IDX]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

print(prompt)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Evaluate \(1 + 2 + 3 + 4 + 5 + 7 + 8 + 9 + 10 \times 0\).<start_working_out>


In [89]:
# Method 1: fast_generate()
response = model.fast_generate(
    prompt,
    # Add other generation parameters if required
)[0]

generated_text = response.outputs[0].text

print(generated_text)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Step 1:  Let's break down the expression step by step.

To evaluate the expression \(1 + 2 + 3 + 4 + 5 + 7 + 8 + 9 + 10 \times 0\), we need to follow the order of operations, which is commonly known as PEMDAS (Parentheses, Exponents, Multiplication and Division, Addition and Subtraction).

Step 2: We have no parentheses or exponents, so let's move to the next step.

Step 3: According to the order of operations, we should perform multiplication and division before addition and subtraction. In our expression, the only multiplication is \(10 \times 0\). So, we first calculate that:
\(10 \times 0 = 0\)

Step 4: Now, we can simplify our expression to \(1 + 2 + 3 + 4 + 5 + 7 + 8 + 9 + 0\).

Step 5: Next, we need to perform addition. We can add the numbers in the expression from left to right:
\[
(1 + 2) + (3 + 4) + (5 + 7) + (8 + 9) + 0 = 3 + 7 + 12 + 17 + 0
\]

Step 6: Now, we add the resulting numbers:
\[
3 + 7 + 12 + 17 + 0 = 39
\]

Step 7: Finally, we have the solution to the expression.

In [90]:
# Method 2: generate() - does not work after GRPO 
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU if available

outputs = model.generate(
    **inputs,
    temperature = 0,
    max_new_tokens=2048,  # Adjust as needed
    # Add other generation parameters if required
)[0]

generated_text = tokenizer.decode(outputs, skip_special_tokens=True)
print(generated_text)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION>Evaluate \(1 + 2 + 3 + 4 + 5 + 7 + 8 + 9 + 10 \times 0\).<start_working_out>A<///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////A<//////////////////////////////////////////////////////////////////////////////////A</////////////////////////////////AA<////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////A</////A<A</////////////////////////////////////////////////////////////////////////A<///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [91]:
# Method 3: text_streamer - does not work after GRPO 
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU if available

_ = model.generate(
    **inputs,
    temperature = 0,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
0\).<start_working_out>A<///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////A<//////////////////////////////////////////////////////////////////////////////////A</////////////////////////////////AA<////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////A</////A<A</////////////////////////////////////////////////////////////////////////A<//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////A</////////////